In [1]:
import pandas as pd
import numpy as np
import dota2api as d2
import re
import requests
import inspect

In [2]:
#team IDs

teams=re.findall('"team_id":(.+?),"', str(requests.get('https://api.opendota.com/api/teams').content))
len(teams)

6431

In [3]:
#Team names

teams_names=re.findall('"name":"(.+?)","', str(requests.get('https://api.opendota.com/api/teams').content))
len(teams_names)

6431

In [4]:
api=d2.Initialise("760552CBCC35C09BF2FB92CC53C42EF6",raw_mode=False) #betterbets.net

In [5]:
# for loop method: [USED METHOD]
#Getting player ID's based on team ID

def player_ids(team):
    team_i=api.get_team_info_by_team_id(start_at_team_id=team, teams_requested=1)
    players=[]
    
    players.append(re.findall("0_account_id': (.+?), '",str(team_i)))
    players.append(re.findall("1_account_id': (.+?), '",str(team_i)))
    players.append(re.findall("2_account_id': (.+?), '",str(team_i)))
    players.append(re.findall("3_account_id': (.+?), '",str(team_i)))
    players.append(re.findall("4_account_id': (.+?), '",str(team_i)))
    players.append(re.findall("5_account_id': (.+?), '",str(team_i)))
    
    flat_list = [item for sublist in players for item in sublist]
    
    return(flat_list)

In [6]:
#list comprehension method (try and fix)

#def player_ids(team):
#    team_i=api.get_team_info_by_team_id(start_at_team_id=team, teams_requested=1)
#    try:
#        return([ item[0] for sublist in [(re.findall("1_account_id': (.+?), '",str(team_i)),
#                            re.findall("2_account_id': (.+?), '",str(team_i)),
#                            re.findall("3_account_id': (.+?), '",str(team_i)),
#                            re.findall("4_account_id': (.+?), '",str(team_i)),
#                           re.findall("5_account_id': (.+?), '",str(team_i)))] for item in sublist])
#    except:
#         return([ item[0] for sublist in [(re.findall("1_account_id': (.+?), '",str(team_i)),
#                            re.findall("2_account_id': (.+?), '",str(team_i)),
#                            re.findall("3_account_id': (.+?), '",str(team_i)),
#                            re.findall("4_account_id': (.+?), '",str(team_i)))] for item in sublist])
#    else:
#        return([ item[0] for sublist in [(re.findall("1_account_id': (.+?), '",str(team_i)),
#                            re.findall("2_account_id': (.+?), '",str(team_i)),
#                            re.findall("3_account_id': (.+?), '",str(team_i)))] for item in sublist])       
#player_ids(teams[0])

In [7]:
#for loop method (works but slow) [USED METHOD]

#players=[]

#for i in range(len(teams)):
#    players.append(player_ids(teams[i]))
#    
#    players_flat=[item for sublist in players for item in sublist]
#    

In [8]:
#players_flat_unique=np.unique(players_flat)
#players_flat_unique=pd.DataFrame(players_flat_unique)
#players_flat_unique.to_csv('players.csv', encoding='utf-8')

players=pd.read_csv('players.csv')
players=players['0'].tolist()

In [9]:
#for loop method (works, but slow)  [USED]
#getting match_ids

#matches=[] 

#for i in range(len(players)):
#    try:
#        matches.append(api.get_match_history(players[i]))
#    except:
#        print("This user hasn't shared their profile", i)

In [10]:
#matchesdf=pd.DataFrame(matches)
#matchesdf.to_csv('matches.csv', encoding='utf-8')

In [11]:
matches=pd.read_csv('matches.csv')
matches=matches['matches'].tolist()

In [12]:
match_ids=[]
for i in range(len(matches)):
    match_ids.append(re.findall("match_id': (.+?), '", matches[i]))
    
match_ids = [item for sublist in match_ids for item in sublist]
match_ids=np.unique(match_ids)

In [13]:
#properly linking item ID to item name.

items=re.findall("\'localized_name\':(.+?), \'",str(str(api.get_game_items())))
item_IDs=re.findall("\'id\': (.+?), \'",str(str(api.get_game_items())))

for i in range(len(items)):
    items[i]=re.sub('^[^a-zA-z]*|[^a-zA-Z]*$','',items[i])
    
item_dict=dict(zip(item_IDs, items))

In [14]:
#properly linking hero ID to hero name.

heroes=re.findall("\'localized_name\':(.+?), \'",str(str(api.get_heroes())))
heroes_IDs=re.findall("\'id\': (.+?), \'",str(str(api.get_heroes())))

for i in range(len(heroes)):
    heroes[i]=re.sub('^[^a-zA-z]*|[^a-zA-Z]*$','',heroes[i])
    
heroes_dict=dict(zip(heroes_IDs, heroes))

In [15]:
#getting match_details for a sample of 10,000

#data_frame=[]

#for i in range(0,10000):
#    data_frame.append(api.get_match_details(match_id=match_ids[i]))

In [16]:
#writing to file.

#f = open("myfile", "w")
#f.write("\n".join(map(lambda x: str(x), data_frame)))
#f.close()

In [17]:
'read in data file produced in the above two cells'
text_file = open("myfile.txt", "r")
data_frame = text_file.read().split("\n")

In [18]:
#fixing up a weird exception where some data was changed. I'm sure there's others, but here's one I noticed
#else, if you want to maintain integrity of the data, change "replacer" to "data_frame" below.

A=[]
D=[]

for i in range(len(data_frame)):
        A.append(str(data_frame[i]).split("\'additional_units\':")[0])
        try:
            D.append(str(data_frame[i]).split("}], \'hero_name\':")[1])
        except:
            D.append("")
            
replacer=[]  

for i in range(len(data_frame)):
    if len(D[i])==0:
        replacer.append(data_frame[i])
    else:
        replacer.append(A[i]+"\'hero_name\':"+D[i])

In [43]:
#seperating match_details into player details.

players=[]

for i in range(len(replacer)):
    for j in range(10):
        try:
            players.append(replacer[i].split("}")[j])
        except:
            players.append("")
            
#in format: player 1(game 1), player 2(game 1),....,player 1(game 2), player 2(game 2),...,

In [44]:
import json
with open('json_sample.txt', 'w') as outfile:
    json.dump(players, outfile)